In [ ]:
# 統合報告書表彰　企業別ランキング集計
# 
# 集計の条件と、各社の集計結果を比較可能なポイントに変換・集計し、企業を順位付けした結果をExcelに出力
# 分析の前提として、抽出に用いた以下のキーワードを各々ワークシートに格納
# ・知的資本関連KW
# ・金融関連KW
# 
# 各社の抽出結果として、以下もワークシートに格納
#・分析対象範囲中の知的資本関連KW
#・同　知的資本関連KW×金融関連KWの共起パターン
#・同　ネットワーク分析の対象となった共起単語中の、知的資本関連KWの中心性指標
# 
# ポイント化する指標は、
# ・抽出された知財関連KWの種類
# ・知財関連KWの件数
# ・知財関連KWの相対的登場頻度
# ・知財関連KW×金融関連KWの共起の種類
# ・知財関連KW×金融関連KW件数
# ・知財関連KW×金融関連KWの相対的登場頻度
# ・金融関連KWの中心性指標
# 
# 点数の付与方法は、
# ・指標ごとの、各企業の順位に対するポイント（上位10社に10点、次の10社に9点、等）※昨年の方式
# ・各企業の指標の数値を比率化して付与するポイント（最大値を100として、結果を 
# 
# 分析対象範囲・点数付与方法毎に、各種ポイントを集約し個別のワークシートに格納
# ワークシート名は、分析対象範囲_点数付与方法’
# 
# 対象KW等の抽出、件数等の集計はGoogleCloud上のBigQueryで実行、結果をDocker上のPythonに取り込み
# 順位付けに使用するポイントはPython上で付与
# 
# リリース
# 2025/5/25 知財ファイナンスWG向けにリリースした内容に対応


In [ ]:
from google.cloud import bigquery
import pandas as pd
import datetime

client = bigquery.Client()
# 対象の分析範囲一覧（全てのSQLテンプレートで共通）
wordlist_table = "my-ip-project-425607.LESJ_Textmining.Merged_WordList"
co_occurrence_table = "my-ip-project-425607.LESJ_Textmining.Merged_Co-Occurrence"
centrality_table = "my-ip-project-425607.LESJ_Textmining.Merged_Centrality"

ip_kw_table = "my-ip-project-425607.LESJ_Textmining.知的資本関連KW"
fn_kw_table = "my-ip-project-425607.LESJ_Textmining.金融関連KW"

target_scopes = ['経営者+価値創造+知的資本', '経営者', '価値創造', '知的資本']


In [ ]:
# 知財関連KWリスト

sql = f"""
SELECT 
    `Keyword`,`count`
FROM 
    `{ip_kw_table}`

ORDER BY `Keyword`
"""

df_ip_kw = client.query(sql).to_dataframe()
df_ip_kw

In [ ]:
# 金融関連KWリスト

sql = f"""
SELECT 
    `Keyword`,`count`
FROM 
     `{fn_kw_table}`
ORDER BY 
    `Keyword`
"""

df_fn_kw = client.query(sql).to_dataframe()
df_fn_kw

In [ ]:
# 1-1. 知財KW一覧・件数リスト

# SQLテンプレート（@target_scope をパラメータ化）
sql = f"""
SELECT
    dt.`コード` AS `コード`,
    dt.`企業名` AS `企業名`,
    dt.`業種` AS `業種`,
    dt.`分析範囲` AS `分析範囲`,
    dt.`単語` AS `知財関連KW`,
    dt.`件数` AS `件数`,
    dt.`件数_Rank` AS `件数_Rank`
FROM
    `{wordlist_table}` dt
LEFT JOIN 
    `{ip_kw_table}` k1
ON 
    dt.`単語` = k1.`Keyword`
WHERE
    k1.keyword IS NOT NULL AND dt.`分析範囲` = @target_scope
ORDER BY
    dt.`コード`, dt.`件数` DESC
"""

# 結果を格納するリスト
results = []

# 各分析範囲ごとにクエリ実行
for scope in target_scopes:
    job_config = bigquery.QueryJobConfig(
        query_parameters=[
            bigquery.ScalarQueryParameter("target_scope", "STRING", scope)
        ]
    )
    df = client.query(sql, job_config=job_config).to_dataframe()

    if df.empty:
        continue  # データがなければスキップ

    # 分析範囲の情報を明示的に追加（念のため）
    # df["指定分析範囲"] = scope

    results.append(df)

# すべての結果を結合
df_ip_kw_list = pd.concat(results, ignore_index=True)

# 結果確認（任意で表示・保存など）
print(df_ip_kw_list.head())



In [ ]:
# 1-2. 知財KW件数の合計にもとづくポイント


# SQL（@target_scope をパラメータとして指定）
sql = f"""
SELECT
    dt.`コード` AS `コード`,
    dt.`企業名` AS `企業名`,
    dt.`分析範囲` AS `分析範囲`,
    SUM(dt.`件数`) AS `総単語件数`
FROM
    `{wordlist_table}` dt
LEFT JOIN 
    `{ip_kw_table}` k1
ON 
    dt.`単語` = k1.`Keyword`
WHERE
    k1.keyword IS NOT NULL AND dt.`分析範囲` = @target_scope
GROUP BY
    `コード`, `企業名`, `分析範囲`
ORDER BY
    `総単語件数` DESC
"""

# 処理結果を格納するリスト
results = []

for scope in target_scopes:
    # パラメータ指定
    job_config = bigquery.QueryJobConfig(
        query_parameters=[
            bigquery.ScalarQueryParameter("target_scope", "STRING", scope)
        ]
    )
    
    # クエリ実行
    df = client.query(sql, job_config=job_config).to_dataframe()

    if df.empty:
        continue  # データがない場合はスキップ

    # Rankを付与
    df['総単語件数_Rank'] = df['総単語件数'].rank(method='dense', ascending=False).astype(int)

    # ランクベースの点数（10点〜1点）
    df['総単語件数_点数'] = (10 - ((df['総単語件数_Rank'] - 1) // 10)).clip(lower=0)

    # 最大値比ベースの点数（100点満点）
    max_count = df['総単語件数'].max()
    df['総単語件数_比率点数'] = (df['総単語件数'] / max_count * 100).round().astype(int)

    # 指定分析範囲列を追加（念のため）
    # df['指定分析範囲'] = scope

    results.append(df)

# 全範囲の結果を結合
df_num_of_ip_kw = pd.concat(results, ignore_index=True)

# 結果確認
print(df_num_of_ip_kw.head())


In [ ]:
# 1-3. 知財KWの種類数にもとづくポイント

# SQLテンプレート（パラメータ化）
sql = f"""
SELECT
    dt.`コード` AS `コード`,
    dt.`企業名` AS `企業名`,
    dt.`分析範囲` AS `分析範囲`,
    COUNT(DISTINCT dt.`単語`) AS `単語種類数`
FROM
    `{wordlist_table}` dt
LEFT JOIN 
    `{ip_kw_table}` k1
ON 
    dt.`単語` = k1.`Keyword`
WHERE
    k1.keyword IS NOT NULL AND dt.`分析範囲` = @target_scope
GROUP BY
    dt.`コード`, dt.`企業名`, dt.`分析範囲`
ORDER BY
    `単語種類数` DESC
"""

# 結果を格納するリスト
results = []

# 各分析範囲について処理
for scope in target_scopes:
    job_config = bigquery.QueryJobConfig(
        query_parameters=[
            bigquery.ScalarQueryParameter("target_scope", "STRING", scope)
        ]
    )
    
    # クエリの実行
    df = client.query(sql, job_config=job_config).to_dataframe()

    if df.empty:
        continue  # データが存在しない場合はスキップ

    # Rank付与
    df['単語種類数_Rank'] = df['単語種類数'].rank(method='dense', ascending=False).astype(int)

    # ランクベース点数（10点〜1点）
    df['単語種類数_点数'] = (10 - ((df['単語種類数_Rank'] - 1) // 10)).clip(lower=0)

    # 最大値比ベースの点数（100点満点）
    max_count = df['単語種類数'].max()
    df['単語種類数_比率点数'] = (df['単語種類数'] / max_count * 100).round().astype(int)

    # 分析範囲情報（念のため追加）
    # df['指定分析範囲'] = scope

    # リストに追加
    results.append(df)

# 全範囲の結果を一括結合
df_num_of_types_of_ip_kw = pd.concat(results, ignore_index=True)

# 出力確認
print(df_num_of_types_of_ip_kw.head())


In [ ]:
# 1-4. 知財KWの相対的重要性（件数ベース）に基づくポイント

# SQL（パラメータ化形式）
sql = f"""
SELECT
    `コード`,
    `企業名`,
    `分析範囲`,
    SUM(`KW_Position_Point`) AS `Total_KW_Position_Point`
FROM (
    SELECT
        dt.`コード` AS `コード`,
        dt.`企業名` AS `企業名`,
        dt.`分析範囲` AS `分析範囲`,
        GREATEST(10 - CAST((dt.`件数_Rank` - 1) / 100 AS INT64), 1) AS `KW_Position_Point`
    FROM
        `{wordlist_table}` dt
    LEFT JOIN 
        `{ip_kw_table}` k1
    ON 
        dt.`単語` = k1.`Keyword`
    WHERE
        k1.keyword IS NOT NULL AND dt.`分析範囲` = @target_scope
)
GROUP BY `コード`, `企業名`, `分析範囲`
ORDER BY `Total_KW_Position_Point` DESC
"""

# 結果格納リスト
results = []

# 各分析範囲に対して実行
for scope in target_scopes:
    job_config = bigquery.QueryJobConfig(
        query_parameters=[
            bigquery.ScalarQueryParameter("target_scope", "STRING", scope)
        ]
    )

    # クエリ実行
    df = client.query(sql, job_config=job_config).to_dataframe()

    if df.empty:
        continue  # 該当データがない場合はスキップ

    # ランク付与
    df['Total_KW_Position_Point_Rank'] = df['Total_KW_Position_Point'].rank(method='dense', ascending=False).astype(int)

    # ランクベースの点数（10点〜1点）
    df['知財kw重要性_点数'] = (10 - ((df['Total_KW_Position_Point_Rank'] - 1) // 10)).clip(lower=0)


    # 最大値比の点数（100点満点）
    max_score = df['Total_KW_Position_Point'].max()
    df['知財kw重要性_比率点数'] = (df['Total_KW_Position_Point'] / max_score * 100).round().astype(int)

    # 範囲識別列を追加
    # df['指定分析範囲'] = scope

    # 結果追加
    results.append(df)

# 結果統合
df_position_of_ip_kw = pd.concat(results, ignore_index=True)

# 出力確認
print(df_position_of_ip_kw.head())



In [ ]:
# 2-2. 知財KW×金融KWの共起件数によるポイント

# SQLテンプレート（@target_scope をパラメータ化）
sql = f"""
SELECT 
    dt.`コード` AS `コード`,
    dt.`企業名` AS `企業名`,
    dt.`分析範囲` AS `分析範囲`,
    SUM(dt.count) AS `共起件数`
FROM 
    `{co_occurrence_table}` dt
LEFT JOIN 
    `{ip_kw_table}` k1
ON 
    dt.`1番目` = k1.keyword OR dt.`2番目` = k1.keyword
LEFT JOIN 
    `my-ip-project-425607.LESJ_Textmining.金融関連KW` k2
ON 
    dt.`1番目` = k2.keyword OR dt.`2番目` = k2.keyword
WHERE
    k1.keyword IS NOT NULL AND k2.keyword IS NOT NULL
    AND dt.`分析範囲` = @target_scope
GROUP BY 
    dt.`コード`, dt.`企業名`, dt.`分析範囲`
ORDER BY 
    `共起件数` DESC
"""

# 結果格納リスト
results = []

# 各分析範囲に対して処理
for scope in target_scopes:
    job_config = bigquery.QueryJobConfig(
        query_parameters=[
            bigquery.ScalarQueryParameter("target_scope", "STRING", scope)
        ]
    )

    # クエリ実行
    df = client.query(sql, job_config=job_config).to_dataframe()

    if df.empty:
        continue  # 該当データがなければスキップ

    # Rankを付与
    df['共起件数_Rank'] = df['共起件数'].rank(method='dense', ascending=False).astype(int)

    # ランクベースの点数（最大10点、最低1点）
    df['共起件数_点数'] = (10 - ((df['共起件数_Rank'] - 1) // 10)).clip(lower=0)

    # 最大値比ベースの点数（最大100点、割合でスコア化）
    max_count = df['共起件数'].max()
    df['共起件数_比率点数'] = (df['共起件数'] / max_count * 100).round().astype(int)

    # 分析範囲識別列を追加（任意）
    # df['指定分析範囲'] = scope

    # 結果追加
    results.append(df)

# すべての分析範囲の結果を統合
df_num_of_co_occurrence = pd.concat(results, ignore_index=True)

# 結果確認
print(df_num_of_co_occurrence.head())


In [ ]:
# 2-1. 知財KW×金融KWの共起リスト

# SQLテンプレート（@target_scope をパラメータ化）
sql = f"""
SELECT 
    dt.`コード` AS `コード`,
    dt.`企業名` AS `企業名`,
    dt.`分析範囲` AS `分析範囲`,
    dt.`1番目` AS `1番目`,
    dt.`2番目` AS `2番目`,
    dt.count,
    dt.count_Rank,
    GREATEST(10 - CAST((dt.count_Rank - 1) / 100 AS INT64), 1) AS `count_Rank_点数`
FROM 
    `{co_occurrence_table}` dt
LEFT JOIN 
    `{ip_kw_table}` k1
ON 
    dt.`1番目` = k1.keyword OR dt.`2番目` = k1.keyword
LEFT JOIN 
    `{fn_kw_table}` k2
ON 
    dt.`1番目` = k2.keyword OR dt.`2番目` = k2.keyword
WHERE
    k1.keyword IS NOT NULL AND k2.keyword IS NOT NULL 
    AND dt.`分析範囲` = @target_scope
ORDER BY
    dt.`コード`, dt.count DESC
"""

# 結果格納リスト
results = []

# 各分析範囲について処理
for scope in target_scopes:
    job_config = bigquery.QueryJobConfig(
        query_parameters=[
            bigquery.ScalarQueryParameter("target_scope", "STRING", scope)
        ]
    )

    # クエリ実行
    df = client.query(sql, job_config=job_config).to_dataframe()

    if df.empty:
        continue  # データがない場合はスキップ

    # df['指定分析範囲'] = scope  # 範囲識別用列を追加
    results.append(df)

# 全分析範囲の結果を統合
df_co_occurrence_list = pd.concat(results, ignore_index=True)

# 表示（開発時用）
print(df_co_occurrence_list.head())


In [ ]:
# 2-3. 知財KW×金融KWの共起種類数によるポイント

# SQLテンプレート（パラメータ化形式）
sql = f"""
SELECT 
    dt.`コード` AS `コード`,
    dt.`企業名` AS `企業名`,
    dt.`分析範囲` AS `分析範囲`,
    COUNT(DISTINCT CONCAT(dt.`1番目`, "-", dt.`2番目`)) AS `共起種類数`
FROM 
    `{co_occurrence_table}` dt
LEFT JOIN 
    `{ip_kw_table}` k1
ON 
    dt.`1番目` = k1.keyword OR dt.`2番目` = k1.keyword
LEFT JOIN 
    `{fn_kw_table}` k2
ON 
    dt.`1番目` = k2.keyword OR dt.`2番目` = k2.keyword
WHERE
    k1.keyword IS NOT NULL AND k2.keyword IS NOT NULL
    AND dt.`分析範囲` = @target_scope
GROUP BY 
    dt.`コード`, dt.`企業名`, dt.`分析範囲`
ORDER BY 
    `共起種類数` DESC
"""

# 結果格納用リスト
results = []

# 各分析範囲についてループ処理
for scope in target_scopes:
    job_config = bigquery.QueryJobConfig(
        query_parameters=[
            bigquery.ScalarQueryParameter("target_scope", "STRING", scope)
        ]
    )

    # クエリ実行
    df = client.query(sql, job_config=job_config).to_dataframe()

    if df.empty:
        continue  # 該当なしならスキップ

    # ランク付与
    df['共起種類数_Rank'] = df['共起種類数'].rank(method='dense', ascending=False).astype(int)

    # ランクベース点数（最大10点、最低1点）
    df['共起種類数_点数'] = (10 - ((df['共起種類数_Rank'] - 1) // 10)).clip(lower=0)

    # 比率ベース点数（最大100点）
    max_val = df['共起種類数'].max()
    df['共起種類数_比率点数'] = (df['共起種類数'] / max_val * 100).round().astype(int)

    # 指定された分析範囲の記録
    # df['指定分析範囲'] = scope

    # 結果追加
    results.append(df)

# すべて統合
df_num_of_types_of_co_occurrence = pd.concat(results, ignore_index=True)

# 結果確認
print(df_num_of_types_of_co_occurrence.head())


In [ ]:
# 2-4. 知財KW×金融KWの相対的重要性（件数ベース）に基づくポイント

# SQL テンプレート（パラメータ化：@target_scope）
sql = f"""
SELECT 
    `コード`,
    `企業名`,
    `分析範囲`,
    SUM(`共起_Position_Point`) AS `Total_共起_Position_Point`
FROM (
    SELECT 
        dt.`コード` AS `コード`,
        dt.`企業名` AS `企業名`,
        dt.`分析範囲` AS `分析範囲`,
        GREATEST(10 - CAST((dt.count_Rank - 1) / 100 AS INT64), 1) AS `共起_Position_Point`
    FROM 
        `{co_occurrence_table}` dt
    LEFT JOIN 
        `{ip_kw_table}` k1
    ON 
        dt.`1番目` = k1.keyword OR dt.`2番目` = k1.keyword
    LEFT JOIN 
        `{fn_kw_table}` k2
    ON 
        dt.`1番目` = k2.keyword OR dt.`2番目` = k2.keyword
    WHERE
        k1.keyword IS NOT NULL AND k2.keyword IS NOT NULL
        AND dt.`分析範囲` = @target_scope
)
GROUP BY
    `コード`, `企業名`, `分析範囲`
ORDER BY
    `Total_共起_Position_Point` DESC
"""

# 結果格納用リスト
results = []

# 各分析範囲で処理を実行
for scope in target_scopes:
    job_config = bigquery.QueryJobConfig(
        query_parameters=[
            bigquery.ScalarQueryParameter("target_scope", "STRING", scope)
        ]
    )

    # クエリ実行
    df = client.query(sql, job_config=job_config).to_dataframe()

    if df.empty:
        continue

    # ランク付与
    df['Total_共起_Position_Point_Rank'] = df['Total_共起_Position_Point'].rank(method='dense', ascending=False).astype(int)

    # ランクベース点数（10点〜1点）
    df['共起kw重要性_点数'] = (10 - ((df['Total_共起_Position_Point_Rank'] - 1) // 10)).clip(lower=0)

    # 比率ベース点数（100点満点換算）
    max_val = df['Total_共起_Position_Point'].max()
    df['共起kw重要性_比率点数'] = (df['Total_共起_Position_Point'] / max_val * 100).round().astype(int)

    # 分析範囲識別列の追加
    # df['指定分析範囲'] = scope

    # 結果を追加
    results.append(df)

# 全範囲結果を統合
df_position_of_co_occurrence = pd.concat(results, ignore_index=True)

# 出力確認
print(df_position_of_co_occurrence.head())



In [ ]:
# 3-1.知財KWの中心性指標一覧

# SQLテンプレート（@target_scope をパラメータ化）
sql = f"""
SELECT
    dt.`コード` AS `コード`,
    dt.`企業名` AS `企業名`,
    dt.`分析範囲` AS `分析範囲`,
    dt.`Node` AS `Node`,
    dt.`Degree Centrality` AS `Degree Centrality`,
    dt.`Betweenness Centrality` AS `Betweenness Centrality`,
    dt.`Closeness Centrality` AS `Closeness Centrality`,
    dt.`Eigenvector Centrality` AS `Eigenvector Centrality`,
    dt.`Community` AS `Community`,
    dt.`Degree Centrality_Rank` AS `Degree Centrality_Rank`,
    dt.`Betweenness Centrality_Rank` AS `Betweenness Centrality_Rank`,
    dt.`Closeness Centrality_Rank` AS `Closeness Centrality_Rank`,
    dt.`Eigenvector Centrality_Rank` AS `Eigenvector Centrality_Rank`
FROM
    `{centrality_table}` dt
LEFT JOIN 
    `{ip_kw_table}` k1
ON 
    dt.`Node` = k1.`Keyword`
WHERE
    k1.keyword IS NOT NULL AND dt.`分析範囲` = @target_scope
ORDER BY
    dt.`コード`, dt.`Degree Centrality` DESC
"""

# 結果格納リスト
results = []

# 各分析範囲に対して処理
for scope in target_scopes:
    job_config = bigquery.QueryJobConfig(
        query_parameters=[
            bigquery.ScalarQueryParameter("target_scope", "STRING", scope)
        ]
    )

    # クエリ実行
    df = client.query(sql, job_config=job_config).to_dataframe()

    if df.empty:
        continue  # データがない場合はスキップ

    # df["指定分析範囲"] = scope  # 範囲識別列を追加
    results.append(df)

# 全範囲結果を統合
df_centrality_list = pd.concat(results, ignore_index=True)

# 結果表示（必要に応じて変更可）
print(df_centrality_list.head())




In [ ]:
# 3-2.知財KWの中心性指標の相対的重要性に基づくポイント


# SQLテンプレート
sql = f"""
SELECT
    `コード`,
    `企業名`,
    `分析範囲`,
    SUM(`Degree Centrality_Position_Point`) AS `Total_Degree Centrality_Position_Point`,
    SUM(`Betweenness Centrality_Position_Point`) AS `Total_Betweenness Centrality_Position_Point`,
    SUM(`Closeness Centrality_Position_Point`) AS `Total_Closeness Centrality_Position_Point`,
    SUM(`Eigenvector Centrality_Position_Point`) AS `Total_Eigenvector Centrality_Position_Point`
FROM (
    SELECT
        dt.`コード`,
        dt.`企業名`,
        dt.`分析範囲`,
        GREATEST(20 - CAST((dt.`Degree Centrality_Rank` - 1) / 10 AS INT64), 1) AS `Degree Centrality_Position_Point`,
        GREATEST(20 - CAST((dt.`Betweenness Centrality_Rank` - 1) / 10 AS INT64), 1) AS `Betweenness Centrality_Position_Point`,
        GREATEST(20 - CAST((dt.`Closeness Centrality_Rank` - 1) / 10 AS INT64), 1) AS `Closeness Centrality_Position_Point`,
        GREATEST(20 - CAST((dt.`Eigenvector Centrality_Rank` - 1) / 10 AS INT64), 1) AS `Eigenvector Centrality_Position_Point`
    FROM
        `{centrality_table}` dt
    LEFT JOIN 
        `{ip_kw_table}` k1
    ON 
        dt.`Node` = k1.`Keyword`
    WHERE
        k1.keyword IS NOT NULL AND dt.`分析範囲` = @target_scope
)
GROUP BY
    `コード`, `企業名`, `分析範囲`
ORDER BY
    `Total_Degree Centrality_Position_Point` DESC
"""

# 結果格納
results = []

for scope in target_scopes:
    job_config = bigquery.QueryJobConfig(
        query_parameters=[
            bigquery.ScalarQueryParameter("target_scope", "STRING", scope)
        ]
    )
    df = client.query(sql, job_config=job_config).to_dataframe()

    if df.empty:
        continue

    # 欠損を0に
    for col in df.columns[3:]:
        df[col] = df[col].fillna(0)

    # 総合スコア列作成
    df["Total_Overall Centrality_Position_Point"] = (
        df["Total_Degree Centrality_Position_Point"] +
        df["Total_Betweenness Centrality_Position_Point"] +
        df["Total_Closeness Centrality_Position_Point"] +
        df["Total_Eigenvector Centrality_Position_Point"]
    )

    # ランクと点数付与
    for metric in [
        "Degree", "Betweenness", "Closeness", "Eigenvector", "Overall"
    ]:
        col = f"Total_{metric} Centrality_Position_Point"
        rank_col = f"{col}_Rank"
        score_col = f"{metric} Centrality_点数"
        ratio_col = f"{metric} Centrality_比率点数"

        df[rank_col] = df[col].rank(method="dense", ascending=False).astype(int)
        df[score_col] = (10 - ((df[rank_col] - 1) // 10)).clip(lower=0)
        max_val = df[col].max()
        df[ratio_col] = (df[col] / max_val * 100).round().astype(int)

    # df["指定分析範囲"] = scope
    results.append(df)

# 結果統合
df_centrality_position_point = pd.concat(results, ignore_index=True)

# 表示

#print(max_count_Degree)

print(df_centrality_position_point)


In [ ]:
data_folder = 'data'
base_folder = 'ir/2025'
info_folder = 'info'
target_info_file = 'target_company_20250513.csv'
target_info_path = data_folder + '/' + target_info_file
df_target_company_info = pd.read_csv(target_info_path)
output_folder = 'output'
# 点数を紐づけるdf（全てのコード・企業を記載したもの）
df_output_list=df_target_company_info[['コード','企業名','業種分類']].sort_values('コード')
# print(df_output_list)

In [ ]:
!pip install XlsxWriter

In [ ]:
import pandas as pd

# 'コード' を文字列として統一（df_output_list およびすべてのスコアデータ）
df_output_list['コード'] = df_output_list['コード'].astype(str)

all_score_dfs = [
    df_num_of_ip_kw,
    df_num_of_types_of_ip_kw,
    df_position_of_ip_kw,
    df_num_of_co_occurrence,
    df_num_of_types_of_co_occurrence,
    df_position_of_co_occurrence,
    df_centrality_position_point
]
for df in all_score_dfs:
    if 'コード' in df.columns:
        df['コード'] = df['コード'].astype(str)

# スコア種別ごとのマッピング（点数, 比率）
rank_score_sources = [
    (df_num_of_ip_kw, '総単語件数_点数'),
    (df_num_of_types_of_ip_kw, '単語種類数_点数'),
    (df_position_of_ip_kw, '知財kw重要性_点数'),
    (df_num_of_co_occurrence, '共起件数_点数'),
    (df_num_of_types_of_co_occurrence, '共起種類数_点数'),
    (df_position_of_co_occurrence, '共起kw重要性_点数'),
    (df_centrality_position_point, 'Overall Centrality_点数'),
    (df_centrality_position_point, 'Degree Centrality_点数'),
    (df_centrality_position_point, 'Betweenness Centrality_点数'),
    (df_centrality_position_point, 'Closeness Centrality_点数'),
    (df_centrality_position_point, 'Eigenvector Centrality_点数')

]

ratio_score_sources = [
    (df_num_of_ip_kw, '総単語件数_比率点数'),
    (df_num_of_types_of_ip_kw, '単語種類数_比率点数'),
    (df_position_of_ip_kw, '知財kw重要性_比率点数'),
    (df_num_of_co_occurrence, '共起件数_比率点数'),
    (df_num_of_types_of_co_occurrence, '共起種類数_比率点数'),
    (df_position_of_co_occurrence, '共起kw重要性_比率点数'),
    (df_centrality_position_point, 'Overall Centrality_比率点数'),
    (df_centrality_position_point, 'Degree Centrality_比率点数'),
    (df_centrality_position_point, 'Betweenness Centrality_比率点数'),
    (df_centrality_position_point, 'Closeness Centrality_比率点数'),
    (df_centrality_position_point, 'Eigenvector Centrality_比率点数')
]

# 合計対象列のリスト（中心性スコアを含む）
# rank_sum_columns = [
#     '総単語件数_点数', '単語種類数_点数', '知財kw重要性_点数',
#     '共起件数_点数', '共起種類数_点数', '共起kw重要性_点数',
#     'Degree Centrality_点数', 'Betweenness Centrality_点数',
#     'Closeness Centrality_点数', 'Eigenvector Centrality_点数', 'Overall Centrality_点数'
# ]
# ratio_sum_columns = [
#     '総単語件数_比率点数', '単語種類数_比率点数', '知財kw重要性_比率点数',
#     '共起件数_比率点数', '共起種類数_比率点数', '共起kw重要性_比率点数',
#     'Degree Centrality_比率点数', 'Betweenness Centrality_比率点数',
#     'Closeness Centrality_比率点数', 'Eigenvector Centrality_比率点数', 'Overall Centrality_比率点数'
# ]
rank_sum_columns = [
     '総単語件数_点数', '単語種類数_点数', '知財kw重要性_点数',
     '共起件数_点数', '共起種類数_点数', '共起kw重要性_点数',
     'Overall Centrality_点数'
]
ratio_sum_columns = [
     '総単語件数_比率点数', '単語種類数_比率点数', '知財kw重要性_比率点数',
     '共起件数_比率点数', '共起種類数_比率点数', '共起kw重要性_比率点数',
     'Overall Centrality_比率点数'
]

# データ統合関数
# スコアデータを集約し、指定列の合計点を追加する関数
def aggregate_scores(score_sources, sum_columns, total_column_name):
        # 分析範囲ごとに処理を行い、結果をまとめる
    combined_df = pd.DataFrame()
    for scope in target_scopes:
                # ベースとなる出力対象企業リストをコピーし、分析範囲を付加
        df_base = df_output_list[['コード', '企業名', '業種分類']].copy()
        df_base['分析範囲'] = scope
                # 各スコア項目について、該当スコープのデータを抽出・マージ
        for df, col in score_sources:
            df = df.copy()  # オリジナルを変更しないようコピー
            if '分析範囲' in df.columns:
                df = df[df['分析範囲'] == scope]  # スコープでフィルタ
            if col in df.columns:
                df = df[['コード', col]]  # 必要な列だけ抽出
                df_base = df_base.merge(df, on='コード', how='left')  # 左結合でスコア追加
                # 指定したスコア列の合計を計算
        df_base[total_column_name] = df_base[sum_columns].sum(axis=1, skipna=True)
                # 各分析範囲の結果を結合
        combined_df = pd.concat([combined_df, df_base], ignore_index=True)
        # 最終的な統合結果を返す
    return combined_df

# 各スコアの統合
rank_df = aggregate_scores(rank_score_sources, rank_sum_columns, '指定点数合計')
ratio_df = aggregate_scores(ratio_score_sources, ratio_sum_columns, '指定比率点数合計')

# ファイル識別に利用する年月日文字列の取得（企業の処理単位で取得）
# 現在の時刻情報を取得
now = datetime.datetime.now()
# 年月日と時刻の文字列を生成
date_time_string = now.strftime("%Y%m%d-%H%M%S")

# Excel出力（分析範囲別シート）
with pd.ExcelWriter(f"{output_folder}/統合点数集計_{date_time_string}.xlsx", engine='xlsxwriter') as writer:
    df_ip_kw.to_excel(writer, sheet_name=f"知財関連KW", index=False)
    df_fn_kw.to_excel(writer, sheet_name=f"金融関連KW", index=False)
    df_ip_kw_list.sort_values(['コード', '分析範囲', '件数'], ascending=[True, True, False]).to_excel(writer, sheet_name=f"知財関連KW抽出リスト", index=False)
    df_co_occurrence_list.sort_values(['コード', '分析範囲', 'count'], ascending=[True, True, False]).to_excel(writer, sheet_name=f"知財KW・金融KW共起抽出リスト", index=False)
    df_centrality_list.sort_values(['コード', '分析範囲', 'Degree Centrality'], ascending=[True, True, False]).to_excel(writer, sheet_name=f"中心性指標リスト", index=False)
    for scope in target_scopes:
        # rank_df[rank_df['分析範囲'] == scope].to_excel(writer, sheet_name=f"{scope}_点数", index=False)
        # ratio_df[ratio_df['分析範囲'] == scope].to_excel(writer, sheet_name=f"{scope}_比率", index=False)
        rank_df[rank_df['分析範囲'] == scope].sort_values('指定点数合計', ascending=False).to_excel(writer, sheet_name=f"{scope}_点数", index=False)
        ratio_df[ratio_df['分析範囲'] == scope].sort_values('指定比率点数合計', ascending=False).to_excel(writer, sheet_name=f"{scope}_比率", index=False)
print(f"{output_folder}/統合点数集計_{date_time_string}.xlsx generated.")